In [1]:
import sys
import re
sys.path.append("/uscms/home/sbrightt/nobackup/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMe/python_analysis/analysisTools")
import plotTools as ptools
import matplotlib.pyplot as plt
import numpy as np
from coffea import hist
import importlib
import os
from analysisTools import fileAnalyzer, Analyzer

ptools.setDefaultStyle()
#plt.style.use(cfgDir+'plt_settings.mplstyle')

In [2]:
histoLoc = "/uscms_data/d3/sbrightt/iDM/iDMe_analysis/CMSSW_10_6_26/src/iDMe/python_analysis/outputs/signal_2018_aEM"
hnames = ["vtx_genMatchByEle","vtx_disp_genMatchByEle","vtx_kin_genMatchByEle","gen_displacement","gen_ee_kinematics","vtx_matchTypeByEle"]
histos = ptools.loadHistosFromFiles(histoLoc,hnames)
info, masses, cts = ptools.getSampleInfo(histos,hname="vtx_genMatchByEle")

In [3]:
# Plotting info about vertices associated with gen matched electron/positron
# (b) vertex chi2
# (c) dR between associated electrons
# (d) dielectron mass
# (e) dielectron energy
# (f) vertex vxy
# (g) vertex vxy significance

reco_vtx_genMatch = histos['vtx_genMatchByEle']
reco_vtx_disp_genMatch = histos['vtx_disp_genMatchByEle']
reco_vtx_kin_genMatch = histos['vtx_kin_genMatchByEle']

sets = ['RL','RLC']
types_bySet = {'RL':['Reg-Reg','Low-Reg','Low-Low','all'],'RLC':['Reg-Reg','Low-Reg','Low-Low','Reg-Cand','Low-Cand','all']}

for pt in list(info.keys()):
    for ct in list(info[pt].keys()):
        for st in sets:
            m, dm = pt.split("-")[0], pt.split("-")[1]
            samp = info[pt][ct]
            outdir = 'plots/{0}-{1}/vtx_matchedByEle/ct-{2}/{3}/'.format(m,dm,ct,st)
            os.makedirs(outdir,exist_ok=True)
            title = r"Gen-Matched $e^+e^-$ Vertices, $m_\chi = {0}$, $\Delta m_\chi = {1}$, $c\tau = {2}$, Set: {3}".format(m,dm,ct,st)

            h_chi2 = reco_vtx_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('dR',overflow='all')
            h_dR = reco_vtx_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('chi2',overflow='all').rebin("dR",2)
            h_vxy = reco_vtx_disp_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('vxy_signif',overflow='all').rebin('vxy',4)
            h_vxy_signif = reco_vtx_disp_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('vxy',overflow='all')
            h_mee = reco_vtx_kin_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('energy',overflow='all')
            h_Eee = reco_vtx_kin_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('mass',overflow='all').rebin('energy',2)
            
            order = ["Reg-Reg","Low-Reg","Low-Low"]
            order_cand = ["Reg-Cand","Low-Cand"]
            
            # plotting vertex reduced chi2
            plt.figure(figsize=(10,8))
            plt.yscale('log')
            hist.plot1d(h_chi2,overlay="type",order=order,density=True)
            plt.xlim([0,20])
            plt.xticks(np.arange(0,21,2))
            plt.title(title)
            plt.savefig(outdir+"RL_chi2_matchedEE_vtx.pdf")
            plt.close()

            # plotting electron dR
            plt.figure(figsize=(10,8))
            #plt.yscale('log')
            hist.plot1d(h_dR,overlay="type",order=order,density=True)
            plt.title(title)
            plt.savefig(outdir+"RL_dR_matchedEE_vtx.pdf")
            plt.close()

            # plotting vertex vxy
            plt.figure(figsize=(10,8))
            plt.yscale('log')
            hist.plot1d(h_vxy,overlay="type",order=order,density=True)
            plt.xlim([0,25])
            plt.title(title)
            plt.savefig(outdir+"RL_vxy_matchedEE_vtx.pdf")
            plt.close()

            # plotting vertex vxy significance
            plt.figure(figsize=(10,8))
            hist.plot1d(h_vxy_signif,overlay="type",order=order,density=True)
            plt.title(title)
            plt.savefig(outdir+"RL_vxySignif_matchedEE_vtx.pdf")
            plt.close()

            # plotting dielectron mass
            plt.figure(figsize=(10,8))
            hist.plot1d(h_mee,overlay="type",order=order,density=True)
            plt.title(title)
            plt.savefig(outdir+"RL_mass_matchedEE_vtx.pdf")
            plt.close()

            # plotting dielectron energy
            plt.figure(figsize=(10,8))
            hist.plot1d(h_Eee,overlay="type",order=order,density=True)
            plt.title(title)
            plt.savefig(outdir+"RL_energy_matchedEE_vtx.pdf")
            plt.close()
            
            if st == "RLC":
                # plotting vertex reduced chi2
                plt.figure(figsize=(10,8))
                plt.yscale('log')
                hist.plot1d(h_chi2,overlay="type",order=order_cand,density=True)
                plt.xlim([0,20])
                plt.xticks(np.arange(0,21,2))
                plt.title(title)
                plt.savefig(outdir+"RLC_chi2_matchedEE_vtx.pdf")
                plt.close()

                # plotting electron dR
                plt.figure(figsize=(10,8))
                #plt.yscale('log')
                hist.plot1d(h_dR,overlay="type",order=order_cand,density=True)
                plt.title(title)
                plt.savefig(outdir+"RLC_dR_matchedEE_vtx.pdf")
                plt.close()

                # plotting vertex vxy
                plt.figure(figsize=(10,8))
                plt.yscale('log')
                hist.plot1d(h_vxy,overlay="type",order=order_cand,density=True)
                plt.xlim([0,25])
                plt.title(title)
                plt.savefig(outdir+"RLC_vxy_matchedEE_vtx.pdf")
                plt.close()

                # plotting vertex vxy significance
                plt.figure(figsize=(10,8))
                hist.plot1d(h_vxy_signif,overlay="type",order=order_cand,density=True)
                plt.title(title)
                plt.savefig(outdir+"RLC_vxySignif_matchedEE_vtx.pdf")
                plt.close()

                # plotting dielectron mass
                plt.figure(figsize=(10,8))
                hist.plot1d(h_mee,overlay="type",order=order_cand,density=True)
                plt.title(title)
                plt.savefig(outdir+"RLC_mass_matchedEE_vtx.pdf")
                plt.close()

                # plotting dielectron energy
                plt.figure(figsize=(10,8))
                hist.plot1d(h_Eee,overlay="type",order=order_cand,density=True)
                plt.title(title)
                plt.savefig(outdir+"RLC_energy_matchedEE_vtx.pdf")
                plt.close()

/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/c

In [4]:
# Same as above, but summing over vertex match types

reco_vtx_genMatch = histos['vtx_genMatchByEle']
reco_vtx_disp_genMatch = histos['vtx_disp_genMatchByEle']
reco_vtx_kin_genMatch = histos['vtx_kin_genMatchByEle']
gen_disp = histos["gen_displacement"]
genEE_kin = histos["gen_ee_kinematics"]

sets = ['RL','RLC']
types_bySet = {'RL':['Reg-Reg','Low-Reg','Low-Low','all'],'RLC':['Reg-Reg','Low-Reg','Low-Low','Reg-Cand','Low-Cand','all']}

for pt in list(info.keys()):
    for ct in list(info[pt].keys()):
        for st in sets:
            m, dm = pt.split("-")[0], pt.split("-")[1]
            samp = info[pt][ct]
            outdir = 'plots/{0}-{1}/vtx_matchedByEle/ct-{2}/{3}/'.format(m,dm,ct,st)
            os.makedirs(outdir,exist_ok=True)
            title = r"Gen-Matched $e^+e^-$ Vertices, $m_\chi = {0}$, $\Delta m_\chi = {1}$, $c\tau = {2}$, Set: {3}".format(m,dm,ct,st)

            h_chi2 = reco_vtx_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('dR','type',overflow='all')
            h_dR = reco_vtx_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('chi2','type',overflow='all').rebin("dR",2)
            h_gen_dR = genEE_kin.integrate("sample",int_range=samp).sum("mass",overflow="all").rebin("dR",2)
            h_vxy = reco_vtx_disp_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('vxy_signif','type',overflow='all').rebin('vxy',4)
            h_gen_vxy = gen_disp.integrate("sample",int_range=samp).sum("vz",overflow="all").rebin("vxy",16)
            h_vxy_signif = reco_vtx_disp_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('vxy','type',overflow='all')
            h_mee = reco_vtx_kin_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('energy','type',overflow='all')
            h_gen_m = genEE_kin.integrate("sample",int_range=samp).sum("dR",overflow="all")
            h_Eee = reco_vtx_kin_genMatch.integrate('sample',int_range=samp).integrate('set',int_range=st).sum('mass','type',overflow='all').rebin('energy',2)

            # plotting vertex reduced chi2
            plt.figure(figsize=(10,8))
            plt.yscale('log')
            hist.plot1d(h_chi2,density=True)
            plt.xlim([0,20])
            plt.xticks(np.arange(0,21,2))
            plt.title(title)
            legend = plt.gca().get_legend()
            legend.get_texts()[0].set_text("Reco Vertices")
            plt.savefig(outdir+"sumTypes_chi2_matchedEE_vtx.pdf")
            plt.close()

            # plotting electron dR
            plt.figure(figsize=(10,8))
            #plt.yscale('log')
            hist.plot1d(h_dR,density=True)
            hist.plot1d(h_gen_dR,density=True)
            plt.title(title)
            legend = plt.gca().get_legend()
            legend.get_texts()[0].set_text("Reco Vertices")
            legend.get_texts()[1].set_text("Gen Vertices")
            plt.savefig(outdir+"sumTypes_dR_matchedEE_vtx.pdf")
            plt.close()

            # plotting vertex vxy
            plt.figure(figsize=(10,8))
            plt.yscale('log')
            hist.plot1d(h_vxy,density=True)
            hist.plot1d(h_gen_vxy,density=True)
            plt.xlim([0,25])
            plt.title(title)
            legend = plt.gca().get_legend()
            legend.get_texts()[0].set_text("Reco Vertices")
            legend.get_texts()[1].set_text("Gen Vertices")
            plt.savefig(outdir+"sumTypes_vxy_matchedEE_vtx.pdf")
            plt.close()

            # plotting vertex vxy significance
            plt.figure(figsize=(10,8))
            hist.plot1d(h_vxy_signif,density=True)
            plt.title(title)
            legend = plt.gca().get_legend()
            legend.get_texts()[0].set_text("Reco Vertices")
            plt.savefig(outdir+"sumTypes_vxySignif_matchedEE_vtx.pdf")
            plt.close()

            # plotting dielectron mass
            plt.figure(figsize=(10,8))
            hist.plot1d(h_mee,density=True)
            hist.plot1d(h_gen_m,density=True)
            plt.title(title)
            legend = plt.gca().get_legend()
            legend.get_texts()[0].set_text("Reco Vertices")
            legend.get_texts()[1].set_text("Gen Vertices")
            plt.savefig(outdir+"sumTypes_mass_matchedEE_vtx.pdf")
            plt.close()

            # plotting dielectron energy
            plt.figure(figsize=(10,8))
            hist.plot1d(h_Eee,density=True)
            plt.title(title)
            legend = plt.gca().get_legend()
            legend.get_texts()[0].set_text("Reco Vertices")
            plt.savefig(outdir+"sumTypes_energy_matchedEE_vtx.pdf")
            plt.close()

/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/c

In [5]:
# plotting match types
reco_vtx_matchType = histos["vtx_matchTypeByEle"]

sets = ['RL','RLC']
types_bySet = {'RL':['Reg-Reg','Low-Reg','Low-Low','all'],'RLC':['Reg-Reg','Low-Reg','Low-Low','Reg-Cand','Low-Cand','all']}

for pt in list(info.keys()):
    for st in sets:
        m, dm = pt.split("-")[0], pt.split("-")[1]
        samp = info[pt][ct]
        samps_ct = [info[pt][ct] for ct in cts]
        outdir = 'plots/ct-overlays/vtx_matchedByEle/{0}-{1}/{2}/'.format(m,dm,st)
        os.makedirs(outdir,exist_ok=True)
        title = r"Gen-Matched $e^+e^-$ Vertices, $m_\chi = {0}$, $\Delta m_\chi = {1}$".format(m,dm)

        h_eff = reco_vtx_matchType.integrate("set",st)

        plt.figure()
        hist.plot1d(h_eff,overlay="sample",order=samps_ct,density=True)
        plt.yscale("log")
        plt.xticks(ticks=np.arange(0.5,7.5,step=1),labels=["No Vertex","Reg-Reg","Low-Reg","Low-Low","Reg-Cand","Low-Cand","Cand-Cand"],fontsize=14)
        plt.ylim([0,1])
        plt.ylabel("Fraction of events",fontsize=14)
        plt.xlabel("")
        plt.title(title)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.savefig(outdir+"gen_vtx_matchType.pdf")
        plt.close()

/tmp/ipykernel_15919/1002525142.py:22: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.ylim([0,1])
/tmp/ipykernel_15919/1002525142.py:22: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.ylim([0,1])
/tmp/ipykernel_15919/1002525142.py:22: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.ylim([0,1])
/tmp/ipykernel_15919/1002525142.py:22: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.ylim([0,1])
/tmp/ipykernel_15919/1002525142.py:22: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.ylim([0,1])
/tmp/ipykernel_15919/1002525142.py:22: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  plt.ylim([0,1])
/tmp/ipyke

In [6]:
# Summed over vertex types, lifetimes overlaid

reco_vtx_genMatch = histos['vtx_genMatchByEle']
reco_vtx_disp_genMatch = histos['vtx_disp_genMatchByEle']
reco_vtx_kin_genMatch = histos['vtx_kin_genMatchByEle']
gen_disp = histos["gen_displacement"]
genEE_kin = histos["gen_ee_kinematics"]

sets = ['RL','RLC']
types_bySet = {'RL':['Reg-Reg','Low-Reg','Low-Low','all'],'RLC':['Reg-Reg','Low-Reg','Low-Low','Reg-Cand','Low-Cand','all']}

for pt in list(info.keys()):
    for st in sets:
        m, dm = pt.split("-")[0], pt.split("-")[1]
        samps_ct = [info[pt][ct] for ct in cts]
        outdir = 'plots/ct-overlays/vtx_matchedByEle/{0}-{1}/{2}/'.format(m,dm,st)
        os.makedirs(outdir,exist_ok=True)
        title = r"Gen-Matched $e^+e^-$ Vertices, $m_\chi = {0}$, $\Delta m_\chi = {1}$, Set: {3}".format(m,dm,ct,st)

        h_chi2 = reco_vtx_genMatch.integrate('set',int_range=st).sum('dR','type',overflow='all')
        h_dR = reco_vtx_genMatch.integrate('set',int_range=st).sum('chi2','type',overflow='all').rebin("dR",2)
        h_gen_dR = genEE_kin.integrate("sample",int_range=samps_ct[0]).sum("mass",overflow="all").rebin("dR",2)
        h_vxy = reco_vtx_disp_genMatch.integrate('set',int_range=st).sum('vxy_signif','type',overflow='all').rebin('vxy',4)
        h_gen_vxy = gen_disp.integrate("sample",int_range=samps_ct[0]).sum("vz",overflow="all").rebin("vxy",16)
        h_vxy_signif = reco_vtx_disp_genMatch.integrate('set',int_range=st).sum('vxy','type',overflow='all')
        h_mee = reco_vtx_kin_genMatch.integrate('set',int_range=st).sum('energy','type',overflow='all')
        h_gen_m = genEE_kin.integrate("sample",int_range=samps_ct[0]).sum("dR",overflow="all")
        h_Eee = reco_vtx_kin_genMatch.integrate('set',int_range=st).sum('mass','type',overflow='all').rebin('energy',2)

        # plotting vertex reduced chi2
        plt.figure(figsize=(10,8))
        plt.yscale('log')
        hist.plot1d(h_chi2,density=True,overlay="sample",order=samps_ct)
        plt.xlim([0,20])
        plt.xticks(np.arange(0,21,2))
        plt.title(title)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.savefig(outdir+"sumTypes_chi2_matchedEE_vtx.pdf")
        plt.close()

        # plotting electron dR
        plt.figure(figsize=(10,8))
        #plt.yscale('log')
        hist.plot1d(h_dR,density=True,overlay="sample",order=samps_ct)
        hist.plot1d(h_gen_dR,density=True)
        plt.title(title)
        legend = plt.gca().get_legend()
        legend.get_texts()[-1].set_text("Gen-Level")
        for txt in legend.get_texts()[:-1]:
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.savefig(outdir+"sumTypes_dR_matchedEE_vtx.pdf")
        plt.close()

        # plotting vertex vxy
        plt.figure(figsize=(10,8))
        plt.yscale('log')
        hist.plot1d(h_vxy,density=True,overlay="sample",order=samps_ct)
        hist.plot1d(h_gen_vxy,density=True)
        plt.xlim([0,25])
        plt.title(title)
        legend = plt.gca().get_legend()
        legend.get_texts()[-1].set_text("Gen-Level")
        for txt in legend.get_texts()[:-1]:
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.savefig(outdir+"sumTypes_vxy_matchedEE_vtx.pdf")
        plt.close()

        # plotting vertex vxy significance
        plt.figure(figsize=(10,8))
        hist.plot1d(h_vxy_signif,density=True,overlay="sample",order=samps_ct)
        plt.title(title)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.savefig(outdir+"sumTypes_vxySignif_matchedEE_vtx.pdf")
        plt.close()

        # plotting dielectron mass
        plt.figure(figsize=(10,8))
        hist.plot1d(h_mee,density=True,overlay="sample",order=samps_ct)
        hist.plot1d(h_gen_m,density=True)
        plt.title(title)
        legend = plt.gca().get_legend()
        legend.get_texts()[-1].set_text("Gen-Level")
        for txt in legend.get_texts()[:-1]:
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.savefig(outdir+"sumTypes_mass_matchedEE_vtx.pdf")
        plt.close()

        # plotting dielectron energy
        plt.figure(figsize=(10,8))
        hist.plot1d(h_Eee,density=True,overlay="sample",order=samps_ct)
        plt.title(title)
        legend = plt.gca().get_legend()
        for txt in legend.get_texts():
            text = txt.get_text()
            txt.set_text(ptools.reduceSampleName(text,lifetime=True))
        plt.savefig(outdir+"sumTypes_energy_matchedEE_vtx.pdf")
        plt.close()

/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/hist/plot.py:333: UserWarning: Attempted to set non-positive bottom ylim on a log-scaled axis.
Invalid limit will be ignored.
  ax.set_ylim(0, None)
/uscms_data/d3/sbrightt/miniconda3/envs/coffea/lib/python3.8/site-packages/c